<a href="https://colab.research.google.com/github/nalderto/POL300-Public/blob/master/modules/module-5/module-5-exercise/module-5-exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 5: Data Analysis Modules

For this activity, you are going to learn about some various modules that allow you to analyze data.  There are going to be a few exercises following each of the different topic sections.  

You will be graded on your code's ability to complete the task described.  All the exercises have test cases to check the functionality of your code.  Make sure the test cases print `OK`.  This means that your code is working fine.  If it doesn't print `OK`, it will tell you which test case is failing.  

To submit this Jupyter notebook to Gradescope, reference the instructions in the module 1 activity or the "Getting Started with Google Colab" guide on Brightspace.

If you are having trouble, don't get discouraged.  Read through the section again.  If you are still having trouble, try to Google around.  There are **TONS** of good online Python resources.  If Google is not being helpful, reach out via the Brightspace Discussion Board.  

## Setup for this Module

This module is intended to prepare you for the module 5 project.  In this project, you will be assigned a specific State of the State address by the governors of each state.  You will scrape the speeches from the respective website, then do some text analysis on the speech.

For this activity, we are going to practice a few text analysis processes using President Trump's 2020 State of the Union address.  To complete this analysis, we are going to be using a few handy Python modules.

## Retrieving the State of the Union Address

When you are going to obtain the state of the state addresses in the module 5 project, you will have to use the data mining techniques that you learned in module 3 (i.e. using the Chrome DevTools to get the HTML tags, Beautiful Soup, etc.)  Since the process was covered in module 3, we are going to gloss over a few of the details.  

1. We start by going to [the website where Trump's 2020 SOTU transcript is hosted](https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-state-union-address-3/). 

2. We open the Chrome DevTools Inspector by right clicking on the text of the transcript and selecting "Inspect".

3. Hover over the HTML code in the sidebar until you find a section that encompasses all the transcript text.  It is okay that it includes some extra text like (U.S. Capitol, Washington, D.C., and 9:06 P.M. EST).  In this case, we are going to use the `div` tag with the classes "page-content_content" and "editor".  
4. Since we want to obtain the transcript text from this section of the HTML.  We use the `.getText()` function on the Beautiful Soup result.
5. As you can see below, we now have the text from Donald Trump's 2020 State of the Union address.  As was mentioned above, there is some extra text included that isn't from the transcript.  If we were really focused on perfection, we would remove the text, however it shouldn't significantly impact our data.

**MAKE SURE YOU RUN THE CODE BELOW, OTHERWISE LATER EXAMPLES WON'T WORK**

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-state-union-address-3/"

page = requests.get(url)

soup = BeautifulSoup(page.content, 'lxml')
transcript_html = soup.find("div", class_="page-content__content editor")

transcript_text = transcript_html.getText()

print(transcript_text)

### Transcript Text Cleaning

We are going to clean up the transcript a little bit.  If you look at the transcript above, you will notice that there are several "(Applause.)", to represent instances where the audience was clapping.  Additionally, the transcript also includes things that the audience were yelling as the indicated by the "AUDIENCE:".  We are going to remove these irregularities so we can focus our analysis on the words that Donald Trump said.

There is a package in Python called `re` that is used for regular expressions (regex).  Regular expressions are a means of defining a sequence of characters.  Much of the functionality of regex is beyond the scope of this class, but we will show you how to use it to filter out the audience comments and the non-spoken content.  

#### Removing Non-spoken Content

Let's first start with removing the non-spoken content.  In this transcript, non-spoken content is contained by parentheses, so want remove everything that is contained by parentheses.  To achieve this, we need to use the `re.sub(pattern, replace, string)` function.  In the `pattern` argument, we need to create a pattern to remove everything that is contained in parentheses.  In this instance, we would use `r'\(.*\)'`

* The `r` means that the string we created is considered a raw string literal.  This essentially allows us to include backslashes in our pattern.  With regular strings, backlashes have other purposes.
* The `'` is simply an apostrophe, which we use when creating a string.
* The `\(` means that our pattern includes an open parenthesis.  We need to "escape" the parenthesis when creating a regular expression, as parentheses are used for grouping patterns in regular expressions.  We basically need to tell regex that we want it to look for a literal open parenthesis in the text.
* The `.*` are two separate things that operate together.  The `.` means to match any character.  The `*` means to match 0 or more instances of the previous character.  Since the previous character is a `.`, then it means that the pattern will match all characters and infinite numbers of times.  
* The `\)` is the same situation as described above.  We are matching the closing parenthesis.

The `replace` argument is the text we would want to replace our pattern matches with.  In this case, it would just be an empty string (`''`), since we want to remove the non-spoken content.  The `string` argument is the text we want to replace, so it would be `transcript_text`.


Removing the non-spoken content contained in parentheses is demonstrated below. 

**MAKE SURE TO RUN THIS CODE SEGMENT TO ENSURE PROPER RESULTS LATER IN THE ACTIVITY**

In [ ]:
import re

transcript_text = re.sub(r'\(.*\)', '', transcript_text)

print(transcript_text)

#### Removing Audience Comments

We can use a similar technique to the the one used to remove non-spoken content in order to remove the audience comments.  In addition to the "AUDIENCE:" speaker ID, want to remove the actual comments.  This can be achieve with the `AUDIENCE:.*\n` pattern.  It will match the word "AUDIENCE:" (with the colon).  Then, it will also match all the text following the speaker ID up to the newline (`\n`).  The newline character (`\n`) signifies the end of the text on that line.  Since all the audience comments are limited to single line in the transcript, we essentially delete an entire line that starts with "AUDIENCE:".  

In order to allow for more accurate textual analysis in a later module, we need to remove extra whitespace (newlines, tabs, spaces).  We want to ensure we only have one space between words and sentences.  We need to first match instances where we have multiple whitespace characters together.  We are using the pattern string `\s+`.  The `\s` represents whitespace characters (newlines, tabs, spaces).  The `+` is similar to the `*`, except it matches one or more instances (instead of zero of more with `*`).  So in this case, when we find one or more whitespace characters, we simply replace them with a single space.  Therefore, our code would be `re.sub(r'\s+', ' ', transcript_text)`.

We also want to remove the instances of `THE PRESIDENT:` speaker ID.  Instead of using regex to achieve this, we are going to use the `.replace` function.  This function is built into every Python string.  It is used like this:

```python
string.replace(old, new)
```
The `string` variable is the string we are modifying.  The `old` argument is the part of the string that we want to replace.  The `new` argument is what `old` is going to be replaced with.  So to get rid of all instances of "THE PRESIDENT:" in the transcript, we would run `transcript_text.replace("THE PRESIDENT:", "")`.  Here, we are simply replacing "THE PRESIDENT:" with an empty string (`""`).

Run the following code to Removing audience comments and the speaker IDs.

**MAKE SURE TO RUN THIS CODE SEGMENT TO ENSURE PROPER RESULTS LATER IN THE ACTIVITY**

In [ ]:
import re

transcript_text = re.sub(r'AUDIENCE:.*\n', '', transcript_text) # Removes audience comments

transcript_text = re.sub(r'\s+', ' ', transcript_text) # Removes extra whitespace

transcript_text = transcript_text.replace('THE PRESIDENT:', '') # Removes THE PRESIDENT speaker ID

print(transcript_text)

## Sentiment Analysis with TextBlob

TextBlob is a Python module that provides easy access to a variety of useful text-processing operations.  Its sentiment analysis tools provide a measurement of polarity and subjectivity.  Polarity is rated on a scale from -1 to 1, where -1 is completely negative and 1 is completely positive (this can be seen in the example below).  Subjectivity is measured on a scale from 0 to 1, where 0 is very objective and 1 is very subjective.

The process to use TextBlob's sentiment analysis functionality, have to take the following steps.

1. Import the TextBlob module with `from textblob import TextBlob`
2. Create an instance of TextBlob with `blob = TextBlob(string)`, where you replace "string" with the text you want to analyze.  This is necessary so TextBlob has the opportunity to process the text.
3. To get the polarity sentiment, type `blob.sentiment.polarity`, where blob is the variable instance of TextBlob that processed your text in step 2.
4. To get the subjectivity sentiment, type `blob.sentiment.subjectivity`, where blob is the variable instance of TextBlob that processed your text in step 2.

In [ ]:
from textblob import TextBlob

positive_text = "WOW!  POL 300 is such a cool class!  I love it so much!"

positive_blob = TextBlob(positive_text)

print("Positive Text's Polarity Score:", positive_blob.sentiment.polarity)


negative_text = "Darn, I am having a really bad day.  All my Python code is broken."

negative_text = TextBlob(negative_text)

print("Negative Text's Polarity Score:", negative_text.sentiment.polarity)


subjective_text = "I believe this class is very important."

subjective_blob = TextBlob(subjective_text)

print("Subjective Text's Subjectivity Score:", subjective_blob.sentiment.subjectivity)


objective_text = "The sky is blue."

objective_blob = TextBlob(objective_text)

print("Objective Text's Subjectivity Score:", objective_blob.sentiment.subjectivity)

## Exercise 1

In this exercise, you will be creating a function to return the polarity and subjectivity scores for Trump's 2020 State of the Union address.  The text from the SOTU address will be passed into the function by the `sotu` argument.  **Your polarity score should be the first argument of your return statement, followed by subjectivity.**

Make sure you ran the "Retrieving the State of the Union Address" code segment earlier in the Python notebook.  Otherwise, you will get an error saying `transcript_text` is not defined.

**HINT: Remember that you can return two variables in a single return statement like so: `return value1, value2`**

In [ ]:
from textblob import TextBlob 

def get_sentiment(sotu):
    
    # Type your code here
    
    
    
    
    
    

# IGNORE BELOW
# Test Cases
import unittest    
    
class TestCases(unittest.TestCase):  
    def test(self):
        polarity, subjectivity = get_sentiment(transcript_text)
        print("Polarity:", polarity)
        print("Subjectivity:", subjectivity)
        self.assertTrue(polarity == 0.15544314121712638)
        self.assertTrue(subjectivity == 0.46102034711905604)


if __name__ == '__main__': 
    unittest.main(argv=[''], exit=False)

## Text Statistics with TextStat

Another really useful Python module for analyzing text is called "TextStat".  The primary purpose of TextStat is to determine the complexity and reading level of a body of text.  TextStat can be imported using: 
```python
import textstat
```

However, before we can import TextStat, we first need to install it.  All of the modules we have worked with thus far have been preinstalled in the Google Colab environment, so we haven't had to worry about installing them.  Luckily, installing modules is as simple as running a single command.  

We will be using *pip* to install this package, which is a package manager for Python modules.  As you continue to use Python, you might come across a package that is useful for your purposes.  Chances are, it isn't preinstalled with your Python environment, so you will need to install it using pip.  To install a Python package in Google Colab, we use the following format:

```python
!pip install package_name
```
So to install TextStat, we would type:

```python
!pip install textstat
```
The exclamation mark tells Google Colab that we are running a command that is not standard to Python.  Then we type "pip" because that is the program we want to run.  Then "install" because we are installing a package.  Finally, we end with the name of the package itself.

And that is it!  After you install a module, you can import it in Python, and use it in your programs.

### Syllable, Word, and Sentence Count

Before we dive into the complexity and reading level statistics, lets first obtain some count statistics.  The text parameter can be replaced with any text content that you want to analyze.

**Make sure you run the cell below, it will install TextStat.  You only need to install a package once per notebook.**  This means you will have to install TextStat again using the same command in later notebooks.

In [ ]:
!pip install textstat # Here we install TextStat
import textstat

text = "This is a sentence. This is another sentence. How amazing is this?"

print(textstat.syllable_count(text)) # Syllabus Count

print(textstat.lexicon_count(text)) # Word Count

print(textstat.sentence_count(text)) # Sentence Count

### Flesch–Kincaid Readability Tests

Flesch–Kincaid is a means of measuring the readability of text.  The TextStat function is `textstat.flesch_reading_ease(text)
`, where `text` is the variable of the measured text.  

Measured texts have a maximum score of 121.22, indicating that it is extremely easy to read.  There is no lower bound, where lower (even negative) scores represent more difficult readability. Here is an example scale:

| Score |    Difficulty     |
|-------|-------------------|
|90-100 | Very Easy         |
| 80-89 | Easy              |
| 70-79 | Fairly Easy       |
| 60-69 | Standard          |
| 50-59 | Fairly Difficult  |
| 30-49 | Difficult         |
| 0-29  | Very Confusing    |

The Flesch–Kincaid ease score can easily be used to estimate the U.S. education system grade level for a text.  The function to get the grade level is `textstat.flesch_kincaid_grade(test)`, where `text` is the variable of the measured text.

In [ ]:
import textstat

simple_text = "I like ball.  Ball is nice."

complex_text = "Her exquisite intellect was thoroughly illistracted in her thesis."

simple_ease = textstat.flesch_reading_ease(simple_text)
simple_grade = textstat.flesch_kincaid_grade(simple_text)

complex_ease = textstat.flesch_reading_ease(complex_text)
complex_grade = textstat.flesch_kincaid_grade(complex_text)


print("Simple Ease Score:", simple_ease)
print("Simple Grade:", simple_grade)
print("Complex Ease Score:", complex_ease)
print("Complex Grade:", complex_grade)

### Gunning Fog Index
TextStat has many different metrics for measuring the number of years of education required to fully understand a text.  Gunning Fog is a metric that uses sentence length and the number of complex words to determine the U.S. grade level needed to comprehend a piece of literature. This metric can be used with the TextStat functions seen below.  Please note that the results may be a little skewed due to the short length of the theses example texts.  

In [ ]:
import textstat

simple_text = "I like ball.  Ball is nice."

complex_text = "Her exquisite intellect was thoroughly illistracted in her thesis."

simple_grade = textstat.gunning_fog(simple_text)
complex_grade = textstat.gunning_fog(complex_text)

print("Simple Grade:", simple_grade)
print("Complex Grade:", complex_grade)

### Automated Readability Index

ARI differs from some other metrics of measuring reading complexity level, as it uses characters instead of syllables.  This choice was made because this index was originally designed to be used for monitoring readability on electric typewriters, since it was easier to keep track of the number of characters instead of the number of syllables ([citation](https://apps.dtic.mil/dtic/tr/fulltext/u2/667273.pdf)).  ARI can be used with the function below:

In [ ]:
import textstat

simple_text = "I like ball.  Ball is nice."

complex_text = "Her exquisite intellect was thoroughly illistracted in her thesis."

simple_grade = textstat.automated_readability_index(simple_text)
complex_grade = textstat.automated_readability_index(complex_text)

print("Simple Grade:", simple_grade)
print("Complex Grade:", complex_grade)

### Coleman-Liau Index

Coleman-Liau is yet another readability index.  It uses the average number of letters per 100 words and average numbers of sentences per 100 words to estimate the U.S. grade level to read a text.  It can be used with the following function:

In [ ]:
import textstat

simple_text = "I like ball.  Ball is nice."

complex_text = "Her exquisite intellect was thoroughly illistracted in her thesis."

simple_grade = textstat.coleman_liau_index(simple_text)
complex_grade = textstat.coleman_liau_index(complex_text)

print("Simple Grade:", simple_grade)
print("Complex Grade:", complex_grade)

### Readability Consensus Score

TextStat has several other metrics of estimating the U.S. grade level needed to understand a text, including Dale-Chall Readability Score, Linsear Write Formula, and the SMOG index.  In an attempt to generate a more accurate grade level, TextStat has a function that aggregates the grade levels from each of these metrics, and generates a single grade from their results.  To generate a readability consensus score, use this function:

```python
textstat.text_standard(text, float_output=True)
```

The `text` variable represents text for analysis.  The `float_output=True` argument, will return the score as a float number, opposed to a string.  We want a number for greater ease during analysis.  

In [ ]:
import textstat

simple_text = "I like ball.  Ball is nice."

complex_text = "Her exquisite intellect was thoroughly illistracted in her thesis."

simple_grade = textstat.text_standard(simple_text, float_output=True)
complex_grade = textstat.text_standard(complex_text, float_output=True)

print("Simple Grade:", simple_grade)
print("Complex Grade:", complex_grade)

## Exercise 2

Here, you are going to use the TextStat module to conduct some analysis of Trump's 2020 State of the Union address.  You are going to return the following values in this order: syllable count, lexicon count, sentence count, Flesch–Kincaid ease score, and Readability Consensus Score (with the `float_output=True` argument).

The State of the Union transcript text is passed into the function below via the `sotu` parameter. 

In [ ]:
import textstat

def get_grade_level(sotu):
    
    # Type your code here
    



    
    
    
    

# IGNORE BELOW
# Test Cases
import unittest    
    
class TestCases(unittest.TestCase):  
    def test(self):
        syllables, words, sentences, flesch, concensus = get_grade_level(transcript_text)
        
        print("Syllables:", syllables)
        print("Words:", words)
        print("Sentences:", sentences)
        print("Flesch–Kincaid Ease Score:", flesch)
        print("Readability Consensus Score:", concensus)
        
        self.assertTrue(syllables == 8160)
        self.assertTrue(words == 5509)
        self.assertTrue(sentences == 331)
        self.assertTrue(flesch == 63.09)
        self.assertTrue(concensus == 9.0)


if __name__ == '__main__': 
    unittest.main(argv=[''], exit=False)

## Keyword Extraction

Keyword extraction will essentially narrow down a document to a few keywords that summarize the topic.  To achieve this, we will be using a module known as Gensim.  To import the keywords functionality from Gensim, use `from gensim.summarization import keywords`.

Then, you will be able to use the function, `keywords(text)`, where `text` is your document.  This function takes a couple arguments that are highly recommended.  The `words=5` argument, will limit the number of keywords to 5.  **This number can be changed to any positive integer.**  This just prevents the list of return keywords from becoming unwieldy.  Especially for longer documents (like State of the Union addresses), the number of returned keywords can be in the dozens.  

The second argument you should use is `split=True`.  This will make the function returns a list of the keywords, instead of just a single string, with spaces separating the words.  This will make it easier for indexing each of the keywords individually.

Finally, `lemmatize=True` will return a single word that represents all the variations of a word.  For instance, "force", "forces", "forced", and "forcing" all essentially mean the same thing, so it would be nice to only have one variation of this word (like "forcing") to encompass all of them.

With all the recommended arguments, the function looks like this:

```python
keywords(text, words=5, split=True, lemmatize=True)
```

## Exercise 3

In this final exercise of module 5, you are going to complete the `get_keywords()` function.  This function is supposed to return a **list of the top 10 keywords** for President Trump's 2020 State of the Union address.  The `sotu` parameter is the text from the State of the Union address.  Use the Summa module to achieve this task.

In [ ]:
from gensim.summarization import keywords

def get_keywords(sotu):
    
    # Type your code here
    
    
    

    
    

# IGNORE BELOW
# Test Cases
import unittest    
    
class TestCases(unittest.TestCase):  
    def test(self):
        words = get_keywords(transcript_text)
        
        print(words)
        
        answer = ['americans', 'new', 'thanks', 'america', 'family', 'tonight', 'free', 'forcing', 'works', 'year']
        
        self.assertTrue(words == answer)


if __name__ == '__main__': 
    unittest.main(argv=[''], exit=False)